<a href="https://colab.research.google.com/github/monjjjjj/Iris-Classification/blob/main/IrisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras import initializers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [97]:
#load data
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
x = dataset[:,0:4].astype('float32')
y = dataset[:,4]
#train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=0)
#print(x.shape)
#print(len(x))
#print(train_x)
#print(len(train_x))
print(y)

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor

In [98]:
#fix random seed for reproducibility
seed = 1
numpy.random.seed(seed)

In [104]:
#encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
#print(encoded_y) 

#convert integers to one-hot vector
one_hot_y = np_utils.to_categorical(encoded_y,3)
train_x, test_x, train_y, test_y = train_test_split(x, one_hot_y, train_size=0.7, test_size=0.3, random_state=0)

In [105]:
#4 inputs: sepal_length, sepal_width, petal_length, petal_width
#3 outputs: Iris Setosa[1, 0, 0], Versicolor[0, 1, 0], or Virginica[0, 0, 1]
def baseline_model():
  model = Sequential()
 
  #model.add(Conv1D(32, 3, activation='relu')(x.shape))
  #model.add(MaxPooling1D(2,data_format='channels_first'))
  #model.add(BatchNormalization())
  

  model.add(Dense(10, kernel_initializer='normal', input_dim=4, activation='sigmoid'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(3, kernel_initializer='normal', activation='softmax'))

  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
  print(model.summary())
  return model

In [101]:
#training
model = baseline_model()
train_history = model.fit(train_x,train_y, epochs=10, batch_size=5, verbose=0)


Model: "sequential_81"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_207 (Dense)           (None, 10)                50        
                                                                 
 dense_208 (Dense)           (None, 6)                 66        
                                                                 
 dense_209 (Dense)           (None, 3)                 21        
                                                                 
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
#k-fold cross validation
'''kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(train_history, x, one_hot_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))'''

'kfold = KFold(n_splits=10, shuffle=True, random_state=seed)\nresults = cross_val_score(train_history, x, one_hot_y, cv=kfold)\nprint("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))'

In [103]:
#evaluation
eval = model.evaluate(test_x, test_y, verbose=0)
print("Evaluation on test data: loss = %0.6f acurracy = %0.2f%% \n") % (eval[0], eval[1] * 100)


Evaluation on test data: loss = %0.6f acurracy = %0.2f%% 



TypeError: ignored